 # Исследование уровня потребительской лояльности  
 
 Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности среди клиентов из России.  


**Цель исследования** — подготовить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков.

**Ход исследования**
 
 Исследование пройдёт в три этапа:
 1. Подключение к базе, выгрузка и предобработка данных.
 2. Создание дашборда в Tableau.
 3. Подготовка презентации.

**Источник данных для анализа:** 
 - результаты NPS-опроса, выгруженные в SQLite.  
 

## Подключение к базе, выгрузка и предобработка данных

In [1]:
import pandas as pd
import os

from sqlalchemy import create_engine

In [2]:
try:
    if os.path.exists('/datasets/telecomm_csi.db'):
        path_to_db = '/datasets/telecomm_csi.db'
    else:
        path_to_db = 'C:/Python_projects/Data_Analyst/Composite_project_2/telecomm_csi.db'
except FileNotFoundError:
    print('Ошибка загрузки! Указанная дирректория или файл не существует!') 

In [3]:
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [4]:
query = """
SELECT *
FROM user
LIMIT 10
"""

In [5]:
pd.read_sql(query, engine)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,34,4,6,8,6
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,55,5,7,8,10
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,28,4,14,5,10
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,41,4,5,8,10
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,5,8,8,9


In [6]:
query = """
SELECT COUNT(*)
FROM user
"""

In [7]:
count_of_rows=pd.read_sql(query, engine)
count_of_rows

,COUNT(*)
0,502493


Проверили соединение с базой данных.  
Посчитали количество записей в таблице `user` для оценки аномальных, пропущенных значениий и  исключения потери данных при сборе "витрины". 

In [8]:
query = """
SELECT MIN(t.lt_day),
       MAX(t.lt_day)
FROM  user AS t
"""

In [9]:
pd.read_sql(query, engine)

,MIN(t.lt_day),MAX(t.lt_day)
0,-21,9162


In [10]:
query = """
SELECT t.age
       
FROM  user AS t
WHERE t.lt_day in (SELECT MAX(t.lt_day)
FROM  user AS t)
"""

In [11]:
pd.read_sql(query, engine)

,age
0,51.0


In [12]:
query = """
SELECT *
FROM  user AS t
WHERE t.lt_day < 0
"""

In [13]:
pd.read_sql(query, engine)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,AEAC2R,-8,None,None,ANDROID,SMARTPHONE,50,8,8,8,1
1,CS0HF8,-2,None,None,ANDROID,SMARTPHONE,15,8,9,8,10
2,DORPT2,-4,None,None,ANDROID,SMARTPHONE,28,8,9,8,4
3,F5O3CG,-21,None,None,ANDROID,SMARTPHONE,60,8,12,8,7
4,LSE939,-13,None,None,ANDROID,SMARTPHONE,22,8,8,8,1
5,QTT7IR,-13,None,None,IOS,SMARTPHONE,28,8,7,8,7
6,RZUS6H,-6,None,None,ANDROID,SMARTPHONE,23,8,11,8,3
7,TYWQW4,-12,None,None,IOS,SMARTPHONE,33,8,5,8,5
8,URKAFI,-11,None,None,ANDROID,SMARTPHONE,30,8,7,8,8
9,V2GNK2,-13,None,None,IOS,SMARTPHONE,38,8,8,8,5


С учетом незначительного количества записей с отрицательным значением "жизни" клиента, а так же с учетом отсутствия по ним данных о возрасте, поле, данные записи будут исключены из запроса (витрины). 

In [14]:
query = """
SELECT MIN(t.age),
       MAX(t.age)
FROM user AS t
"""

In [15]:
pd.read_sql(query, engine)

,MIN(t.age),MAX(t.age)
0,10.0,89.0


In [16]:
query = """
SELECT *
FROM user AS t
WHERE t.age IS NULL
"""

In [17]:
pd.read_sql(query, engine)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A1E59W,4055,None,NaN,ANDROID,SMARTPHONE,13,8,5,8,1
1,A28ZDT,4243,None,NaN,ANDROID,SMARTPHONE,56,8,5,8,5
2,A2GLPQ,2354,None,NaN,ANDROID,SMARTPHONE,16,8,7,8,1
3,A41C1K,557,None,NaN,ANDROID,SMARTPHONE,38,8,5,6,3
4,A513VG,4705,None,NaN,IOS,SMARTPHONE,13,8,24,8,9
...,...,...,...,...,...,...,...,...,...,...,...
549,ZUDYU9,520,None,NaN,ANDROID,SMARTPHONE,41,8,25,6,9
550,ZWQJRW,220,None,0.0,ANDROID,SMARTPHONE,40,8,4,5,1
551,ZXK9AK,3550,None,NaN,ANDROID,SMARTPHONE,50,8,5,8,4
552,ZZRS2G,345,None,0.0,ANDROID,SMARTPHONE,28,8,5,5,5


Со значениями возраста аномалий нет. 554 строки не содержат возраст - это порядка 0,1% от общего объема данных. Данные записи будуд исключены из итогового запроса.

In [18]:
query = '''SELECT distinct t.gender_segment
              FROM user AS t'''

In [19]:
pd.read_sql(query, engine)

,gender_segment
0,1.0
1,0.0
2,NaN


In [20]:
query = '''SELECT t.gender_segment,
                  COUNT(*) AS cnt  
           FROM user AS t
           GROUP BY 1
           ORDER BY 2 DESC'''

In [21]:
pd.read_sql(query, engine)

,gender_segment,cnt
0,1.0,272442
1,0.0,228750
2,NaN,1301


1301 запись без указания пола.

In [22]:
query = '''SELECT distinct t.os_name
              FROM user AS t'''

In [23]:
pd.read_sql(query, engine)

,os_name
0,ANDROID
1,IOS
2,OTHER
3,unknown
4,PROPRIETARY
5,WINDOWS PHONE
6,SYMBIAN OS
7,BADA OS
8,WINDOWS MOBILE


In [24]:
query = '''SELECT t.os_name,
                  COUNT(*) AS cnt  
           FROM user AS t
           GROUP BY 1
           ORDER BY 2 DESC'''

In [25]:
pd.read_sql(query, engine)

,os_name,cnt
0,ANDROID,435981
1,IOS,60051
2,PROPRIETARY,2491
3,OTHER,2394
4,WINDOWS PHONE,1297
5,unknown,133
6,SYMBIAN OS,103
7,BADA OS,40
8,WINDOWS MOBILE,3


In [26]:
query = '''SELECT distinct t.cpe_type_name
              FROM user AS t'''

In [27]:
pd.read_sql(query, engine)

,cpe_type_name
0,SMARTPHONE
1,TABLET
2,MOBILE PHONE/FEATURE PHONE
3,PHONE
4,ROUTER
5,MODEM
6,WLAN ROUTER
7,USB MODEM
8,unknown
9,PORTABLE(INCLUDE PDA)


In [28]:
query = '''SELECT distinct t.country
              FROM location AS t'''

In [29]:
pd.read_sql(query, engine)

,country
0,Россия


Все клиенты действительно из России.

In [30]:
query = '''SELECT distinct t.nps_score
              FROM user AS t'''

In [31]:
pd.read_sql(query, engine)

,nps_score
0,10
1,6
2,9
3,2
4,5
5,8
6,1
7,4
8,7
9,3


Аномальных значений оценок клиентов в NPS-опросе нет.

In [32]:
query = """
WITH
t AS (SELECT *,
       CASE 
           WHEN lt_day <= 365 THEN 'да'
           ELSE 'нет'
       END AS is_new,

       CASE 
           WHEN gender_segment = 1 THEN 'женщина'
           WHEN gender_segment = 0 THEN 'мужчина'
           ELSE 'нет данных'
       END AS gen_segment,
       
       CASE 
           WHEN nps_score <= 6 THEN 'критики'
           WHEN nps_score >= 9 THEN 'сторонники'
           ELSE 'нейтралы'
       END AS nps_group
      
FROM user)
SELECT t.user_id,
       t.lt_day,
       t.is_new,
       t.age,
       t.gen_segment AS gender_segment,
       t.nps_score,
       t.nps_group,
       t.os_name,
       t.cpe_type_name,
       loc.country,
       loc.city,
       SUBSTR(age.title,4) AS age_segment,
       SUBSTR(traf.title,4) AS traffic_segment,
       SUBSTR(life.title,4) AS lifetime_segment
       
       
FROM t LEFT OUTER JOIN location AS loc ON t.location_id = loc.location_id
LEFT OUTER JOIN age_segment AS age ON t.age_gr_id = age.age_gr_id
LEFT OUTER JOIN traffic_segment AS traf ON t.tr_gr_id = traf.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS life ON t.lt_gr_id = life.lt_gr_id
WHERE t.lt_day >= 0 AND t.age IS NOT NULL

LIMIT 5"""

In [33]:
pd.read_sql(query, engine)

,user_id,lt_day,is_new,age,gender_segment,nps_score,nps_group,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,нет,45.0,женщина,10,сторонники,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+
1,A001WF,2344,нет,53.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+
2,A003Q7,467,нет,57.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24
3,A004TB,4190,нет,44.0,женщина,10,сторонники,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+
4,A004XT,1163,нет,24.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+


Head "витрины" не вызывает ошибок, делаем полный запрос.

In [34]:
query = """
WITH
t AS (SELECT *,
       CASE 
           WHEN lt_day <= 365 THEN 'да'
           ELSE 'нет'
       END AS is_new,

       CASE 
           WHEN gender_segment = 1 THEN 'женщина'
           WHEN gender_segment = 0 THEN 'мужчина'
           ELSE 'нет данных'
       END AS gen_segment,
       
       CASE 
           WHEN nps_score <= 6 THEN 'критики'
           WHEN nps_score >= 9 THEN 'сторонники'
           ELSE 'нейтралы'
       END AS nps_group
      
FROM user)
SELECT t.user_id,
       t.lt_day,
       t.is_new,
       t.age,
       t.gen_segment AS gender_segment,
       t.nps_score,
       t.nps_group,
       t.os_name,
       t.cpe_type_name,
       loc.country,
       loc.city,
       SUBSTR(age.title,4) AS age_segment,
       SUBSTR(traf.title,4) AS traffic_segment,
       SUBSTR(life.title,4) AS lifetime_segment
       
       
FROM t LEFT OUTER JOIN location AS loc ON t.location_id = loc.location_id
LEFT OUTER JOIN age_segment AS age ON t.age_gr_id = age.age_gr_id
LEFT OUTER JOIN traffic_segment AS traf ON t.tr_gr_id = traf.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS life ON t.lt_gr_id = life.lt_gr_id
WHERE t.lt_day >= 0 AND t.age IS NOT NULL
"""

In [35]:
telecomm = pd.read_sql(query, engine)

In [36]:
telecomm.head()

,user_id,lt_day,is_new,age,gender_segment,nps_score,nps_group,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,нет,45.0,женщина,10,сторонники,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+
1,A001WF,2344,нет,53.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+
2,A003Q7,467,нет,57.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24
3,A004TB,4190,нет,44.0,женщина,10,сторонники,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+
4,A004XT,1163,нет,24.0,мужчина,10,сторонники,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+


In [37]:
telecomm.shape

(501939, 14)

In [38]:
count_of_rows

,COUNT(*)
0,502493


Данные при конструировании "витрины" не утеряны. Датафрейм содержит 501939 строк, что меньше начальной таблицы на 554 строки без указания возраста (в том числе 13  с отрицательным значением "жизни" клиента). Витрина содержит требуемые 14 колонок.

In [39]:
telecomm.to_csv('telecomm.csv', index=False)

## Дашборд  

Ссылка на [дашборд][1] 

[1]:https://public.tableau.com/app/profile/stepankl/viz/Telecomm_project_2/sheet11?publish=yes     "дашборд"


## Презентация  

Ссылка на [презентацию][2] 

[2]:https://drive.google.com/file/d/1E36gb6YgRrBaGSd1GxigHvcVRYGSFUOC/view?usp=sharing   "презентацию"


## Выводы

* Распределение участников опроса по возрастным группам:
35-44  - 33,06 %
25-34  - 30,84 %
45-54  - 18,78 %
55-64  -   8,58 %
16-24  -   5,97 %
66 +    -   2,64 %
0- 16   -   0,13 % 

* Во всех возрастных группах, за исключением категории 16-24, преобладают респонденты женского пола.

* Старых клиентов значительно больше (на 65,74%). С увеличением возраста доля старых клиентов увеличивается. Влияние пола на  данный показатель не существенное –  доля старых клиентов среди женщин на 1,75 % больше.

* Больше всего респондентов из Москвы – 13,16%. Респонденты из первых пяти городов (Москва, Санкт-Петербург, Новосибирск, Екатеринбург и Казань) составили третью часть всех опрошенных – 33,4%.

* Вне зависимости от пола и возрастной категории с увеличением срока жизни клиента лояльность уменьшается.  
Более того, среди клиентов женского пола в возрасте от 16 до 24 лет после 36 месяцев пользования «продуктом» наблюдается отрицательный NPS,среди клиентов мужского пола в возрасте от 16 до 24 лет NPS отрицательный уже после 24 месяцев пользования продуктом. 
* Среди клиентов в возрасте до 55 лет наблюдается снижение лояльности с увеличением трафика.
* Наименьшая удовлетворенность наблюдается у пользователей с операционной системой IOS .
* Топ 5 городов по лояльности: Иваново, Череповец, Саранск, Волжский, Новокузнецк.
* Аутсайдеры: Воронеж, Тольятти, Рязань, Тверь, Тюмень. 
* Общий  NPS =     +21,98
* Клиенты, которые относятся к группе сторонников – клиенты в возрасте до 44 лет с сроком жизни до 24 месяцев и невысоким трафиком и клиенты в возрасте старше 45 лет в не зависимости от срока жизни и трафика.
К критикам можно отнести женщин в возрасте до 24 лет и мужчин в возрасте до 34 лет сроком жизни более 36 месяцев, а также пользователей IOS в возрасте 34 лет. 
 


## Чек-лист готовности проекта

- [x]  подключиться к базе
- [x]  данные изучены 
- [x]  данные собраны в одну таблицу   
- [x]  таблица сохранена как csv-файл 
- [x]  разработан макет дашборда
- [x]  дашборд построен  
- [x]  с помощью дашборда даны ответы на поставленные вопросы  
- [x]  подготовлена презентация 